# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [12]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess


In [13]:
load_dotenv(override=True)

# load keys
google_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:6]}")
else:
    print("Google API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:6]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")


Google API Key exists and begins AIzaSy
Groq API Key exists and begins gsk_Hl
OpenRouter API Key exists and begins sk-or-


In [53]:

# A thin wrapper around calls to HTTP endpoints



# For Gemini, DeepSeek and Groq, we can use the OpenAI python client
# Because Google and DeepSeek have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url
from openai import OpenAI



gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [58]:
models = [
    "openai/gpt-oss-120b",          # Google free tier
    "qwen2.5-coder",             # Ollama local
    "openai/gpt-oss-120b",         # Groq free
    "qwen/qwen3-coder-30b-a3b-instruct",           # OpenRouter free
]
clients = {
    "openai/gpt-oss-120b": gemini,
    "qwen2.5-coder": ollama,
    "openai/gpt-oss-120b": groq,
    "qwen/qwen3-coder-30b-a3b-instruct": openrouter,
}


In [16]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Linux',
  'arch': 'x86_64',
  'release': '6.14.0-37-generic',
  'version': '#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2',
  'kernel': '6.14.0-37-generic',
  'distro': {'name': 'Ubuntu 24.04.2 LTS', 'version': '24.04'},
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'x86_64-linux-gnu'},
 'package_managers': ['apt'],
 'cpu': {'brand': 'Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz',
  'cores_logical': 8,
  'cores_physical': 4,
  'simd': ['AVX', 'AVX2', 'AVX512F', 'FMA', 'SSE4_2']},
 'toolchain': {'compilers': {'gcc': 'gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0',
   'g++': 'g++ (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 4.3'},
  'linkers': {'ld_lld': ''}}}

### Use to run genrated code 
 https://www.programiz.com/cpp-programming/online-compiler/

In [17]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]


## And now, on with the main task

In [18]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [19]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [20]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [63]:
# def port(model, python):
#     client = clients[model]
#     reasoning_effort = "high" if model in ["qwen3-coder-30b", "openai/gpt-oss-120b"] else "normal"
#     response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
#     reply = response.choices[0].message.content
#     reply = reply.replace('```cpp','').replace('```','')
#     write_output(reply)
#     return reply


def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if model in ["qwen3-coder-30b", "openai/gpt-oss-120b"] else "normal"

    if model.startswith("gemini"):
        response = client.generate_content(messages_for(python))
        reply = response.text
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages_for(python)
        )
        reply = response.choices[0].message.content

    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply


In [47]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [66]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        model = gr.Dropdown(models, label="Select model", value=models[0])
        convert = gr.Button("Convert code")

    convert.click(port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


### excution time

Qwen 2.5 Coder: Fail  
(Groq)OpenAI gpt-oss-120B: 1.513213  
(Gemini)OpenAI gpt-oss 120B: 1.520153  
Qwen 30B: 1.775681